# CUDA Fluid Simulation with Interactive Visualization

## 1. Package Installation

In [286]:
!pip install pycuda ipywidgets ipyevents rich

## 2. Imports and Initial Setup

In [287]:
# Import libraries
import numpy as np
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import time
from IPython.display import clear_output, display, HTML
import ipywidgets as widgets
from rich.console import Console
from rich.panel import Panel
from rich.text import Text
from rich import box
import io

# Add custom CSS for black text in buttons
# Add custom CSS for black text in buttons
display(HTML("""
<style>
.widget-button {
    color: black !important;
    font-weight: bold !important;
}
.widget-button .widget-button-text {
    color: black !important;
}
.jupyter-widgets .widget-button {
    color: black !important;
}
.jupyter-widgets button {
    color: black !important;
}
/* Force black text on all button states */
.widget-button:hover, .widget-button:focus, .widget-button:active {
    color: black !important;
}
</style>
"""))

## 3. Simulation Parameters

In [288]:
# Simulation parameters
N = 80  # Grid size (smaller for ASCII display)
dt = 0.1  # Time step
diff = 0.0001  # Diffusion rate
visc = 0.0001  # Viscosity
force = 5.0  # Force multiplier
source = 100.0  # Density source strength

## 4. CUDA Kernel Code

In [289]:
# CUDA code for fluid simulation
cuda_code = """
#define IX(x, y) ((x) + (y) * N)
#define TILE_SIZE 16

// Add source
__global__ void add_source(float *x, float *s, float dt, int N) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < N * N) {
        x[i] += dt * s[i];
    }
}

// Diffuse
__global__ void diffuse(float *x, float *x0, float diff, float dt, int b, int N) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    int j = blockIdx.y * blockDim.y + threadIdx.y;
    int idx = IX(i, j);

    if (i > 0 && i < N-1 && j > 0 && j < N-1) {
        float a = dt * diff * N * N;
        x[idx] = (x0[idx] + a * (x0[IX(i-1, j)] + x0[IX(i+1, j)] +
                               x0[IX(i, j-1)] + x0[IX(i, j+1)])) / (1 + 4*a);
    }
}

// Advect
__global__ void advect(float *d, float *d0, float *u, float *v, float dt, int b, int N) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    int j = blockIdx.y * blockDim.y + threadIdx.y;

    if (i < N && j < N) {
        int idx = IX(i, j);

        if (i > 0 && i < N-1 && j > 0 && j < N-1) {
            // Trace particle back
            float x = i - dt * N * u[idx];
            float y = j - dt * N * v[idx];

            // Clamp values
            if (x < 0.5f) x = 0.5f;
            if (x > N - 1.5f) x = N - 1.5f;
            if (y < 0.5f) y = 0.5f;
            if (y > N - 1.5f) y = N - 1.5f;

            // Interpolation indices
            int i0 = (int)x;
            int i1 = i0 + 1;
            int j0 = (int)y;
            int j1 = j0 + 1;

            // Interpolation weights
            float s1 = x - i0;
            float s0 = 1 - s1;
            float t1 = y - j0;
            float t0 = 1 - t1;

            // Bilinear interpolation
            d[idx] = s0 * (t0 * d0[IX(i0, j0)] + t1 * d0[IX(i0, j1)]) +
                     s1 * (t0 * d0[IX(i1, j0)] + t1 * d0[IX(i1, j1)]);
        }
    }
}

// Project step 1: Compute divergence
__global__ void project1(float *u, float *v, float *p, float *div, int N) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    int j = blockIdx.y * blockDim.y + threadIdx.y;

    if (i > 0 && i < N-1 && j > 0 && j < N-1) {
        int idx = IX(i, j);
        div[idx] = -0.5f * (
            u[IX(i+1, j)] - u[IX(i-1, j)] +
            v[IX(i, j+1)] - v[IX(i, j-1)]
        ) / N;
        p[idx] = 0;
    }
}

// Project step 2: Solve pressure
__global__ void project2(float *p, float *div, int N) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    int j = blockIdx.y * blockDim.y + threadIdx.y;

    if (i > 0 && i < N-1 && j > 0 && j < N-1) {
        int idx = IX(i, j);
        p[idx] = (div[idx] + p[IX(i-1, j)] + p[IX(i+1, j)] +
                p[IX(i, j-1)] + p[IX(i, j+1)]) / 4.0f;
    }
}

// Project step 3: Apply pressure forces
__global__ void project3(float *u, float *v, float *p, int N) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    int j = blockIdx.y * blockDim.y + threadIdx.y;

    if (i > 0 && i < N-1 && j > 0 && j < N-1) {
        int idx = IX(i, j);
        u[idx] -= 0.5f * N * (p[IX(i+1, j)] - p[IX(i-1, j)]);
        v[idx] -= 0.5f * N * (p[IX(i, j+1)] - p[IX(i, j-1)]);
    }
}

// Set boundary conditions
__global__ void set_bnd(float *x, int b, int N) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;

    if (i < N) {
        // Set values at top and bottom edges
        x[IX(i, 0)] = b == 2 ? -x[IX(i, 1)] : x[IX(i, 1)];
        x[IX(i, N-1)] = b == 2 ? -x[IX(i, N-2)] : x[IX(i, N-2)];

        // Set values at left and right edges
        x[IX(0, i)] = b == 1 ? -x[IX(1, i)] : x[IX(1, i)];
        x[IX(N-1, i)] = b == 1 ? -x[IX(N-2, i)] : x[IX(N-2, i)];
    }

    // Set values at corners (only done by thread 0)
    if (i == 0) {
        x[IX(0, 0)] = 0.5f * (x[IX(1, 0)] + x[IX(0, 1)]);
        x[IX(0, N-1)] = 0.5f * (x[IX(1, N-1)] + x[IX(0, N-2)]);
        x[IX(N-1, 0)] = 0.5f * (x[IX(N-2, 0)] + x[IX(N-1, 1)]);
        x[IX(N-1, N-1)] = 0.5f * (x[IX(N-2, N-1)] + x[IX(N-1, N-2)]);
    }
}

// Apply forces
__global__ void apply_force(float *u, float *v, float *d, int forceX, int forceY,
                           float dirX, float dirY, float density, int radius, int N) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    int j = blockIdx.y * blockDim.y + threadIdx.y;

    if (i < N && j < N) {
        int idx = IX(i, j);

        float dx = i - forceX;
        float dy = j - forceY;
        float dist = sqrtf(dx*dx + dy*dy);

        if (dist < radius) {
            float factor = 1.0f - dist/radius;

            // Add density
            d[idx] += density * factor;

            // Add velocity
            u[idx] += dirX * factor;
            v[idx] += dirY * factor;
        }
    }
}

// Clear all fields
__global__ void clear_fields(float *u, float *v, float *d, int N) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < N * N) {
        u[i] = 0.0f;
        v[i] = 0.0f;
        d[i] = 0.0f;
    }
}
"""

## 5. CUDA Compilation and Setup

In [290]:
# Compile CUDA kernels
mod = SourceModule(cuda_code)

# Get kernel functions
add_source_kernel = mod.get_function("add_source")
diffuse_kernel = mod.get_function("diffuse")
advect_kernel = mod.get_function("advect")
project1_kernel = mod.get_function("project1")
project2_kernel = mod.get_function("project2")
project3_kernel = mod.get_function("project3")
set_bnd_kernel = mod.get_function("set_bnd")
apply_force_kernel = mod.get_function("apply_force")
clear_fields_kernel = mod.get_function("clear_fields")

## 6. Memory Allocation and Initialization

In [291]:
# Initialize arrays
dens = np.zeros((N, N), dtype=np.float32)
dens_prev = np.zeros((N, N), dtype=np.float32)
u = np.zeros((N, N), dtype=np.float32)
v = np.zeros((N, N), dtype=np.float32)
u_prev = np.zeros((N, N), dtype=np.float32)
v_prev = np.zeros((N, N), dtype=np.float32)

# Allocate memory on GPU
d_dens = cuda.mem_alloc(dens.nbytes)
d_dens_prev = cuda.mem_alloc(dens_prev.nbytes)
d_u = cuda.mem_alloc(u.nbytes)
d_v = cuda.mem_alloc(v.nbytes)
d_u_prev = cuda.mem_alloc(u_prev.nbytes)
d_v_prev = cuda.mem_alloc(v_prev.nbytes)
d_p = cuda.mem_alloc(dens.nbytes)
d_div = cuda.mem_alloc(dens.nbytes)

# Copy initial data to device
cuda.memcpy_htod(d_dens, dens)
cuda.memcpy_htod(d_dens_prev, dens_prev)
cuda.memcpy_htod(d_u, u)
cuda.memcpy_htod(d_v, v)
cuda.memcpy_htod(d_u_prev, u_prev)
cuda.memcpy_htod(d_v_prev, v_prev)

# Thread block and grid dimensions
block_2d = (16, 16, 1)
grid_2d = ((N + block_2d[0] - 1) // block_2d[0],
          (N + block_2d[1] - 1) // block_2d[1])

block_1d = (256, 1, 1)
grid_1d = ((N + block_1d[0] - 1) // block_1d[0], 1)

## 7. Simulation Core Functions

In [292]:
def vel_step():
    # Diffuse velocities
    cuda.memcpy_dtod(d_u_prev, d_u, u.nbytes)
    cuda.memcpy_dtod(d_v_prev, d_v, v.nbytes)

    for _ in range(20):
        diffuse_kernel(d_u, d_u_prev, np.float32(visc), np.float32(dt), np.int32(1), np.int32(N),
                      block=block_2d, grid=grid_2d)
        set_bnd_kernel(d_u, np.int32(1), np.int32(N), block=block_1d, grid=grid_1d)

    for _ in range(20):
        diffuse_kernel(d_v, d_v_prev, np.float32(visc), np.float32(dt), np.int32(2), np.int32(N),
                      block=block_2d, grid=grid_2d)
        set_bnd_kernel(d_v, np.int32(2), np.int32(N), block=block_1d, grid=grid_1d)

    # Project to ensure mass conservation
    project1_kernel(d_u, d_v, d_p, d_div, np.int32(N), block=block_2d, grid=grid_2d)

    for _ in range(20):
        project2_kernel(d_p, d_div, np.int32(N), block=block_2d, grid=grid_2d)
        set_bnd_kernel(d_p, np.int32(0), np.int32(N), block=block_1d, grid=grid_1d)

    project3_kernel(d_u, d_v, d_p, np.int32(N), block=block_2d, grid=grid_2d)
    set_bnd_kernel(d_u, np.int32(1), np.int32(N), block=block_1d, grid=grid_1d)
    set_bnd_kernel(d_v, np.int32(2), np.int32(N), block=block_1d, grid=grid_1d)

    # Advect velocities
    cuda.memcpy_dtod(d_u_prev, d_u, u.nbytes)
    cuda.memcpy_dtod(d_v_prev, d_v, v.nbytes)

    advect_kernel(d_u, d_u_prev, d_u_prev, d_v_prev, np.float32(dt), np.int32(1), np.int32(N),
                 block=block_2d, grid=grid_2d)
    set_bnd_kernel(d_u, np.int32(1), np.int32(N), block=block_1d, grid=grid_1d)

    advect_kernel(d_v, d_v_prev, d_u_prev, d_v_prev, np.float32(dt), np.int32(2), np.int32(N),
                 block=block_2d, grid=grid_2d)
    set_bnd_kernel(d_v, np.int32(2), np.int32(N), block=block_1d, grid=grid_1d)

    # Project again
    project1_kernel(d_u, d_v, d_p, d_div, np.int32(N), block=block_2d, grid=grid_2d)

    for _ in range(20):
        project2_kernel(d_p, d_div, np.int32(N), block=block_2d, grid=grid_2d)
        set_bnd_kernel(d_p, np.int32(0), np.int32(N), block=block_1d, grid=grid_1d)

    project3_kernel(d_u, d_v, d_p, np.int32(N), block=block_2d, grid=grid_2d)
    set_bnd_kernel(d_u, np.int32(1), np.int32(N), block=block_1d, grid=grid_1d)
    set_bnd_kernel(d_v, np.int32(2), np.int32(N), block=block_1d, grid=grid_1d)

def dens_step():
    # Add source
    add_source_kernel(d_dens, d_dens_prev, np.float32(dt), np.int32(N),
                    block=block_2d, grid=grid_2d)

    # Swap for diffusion
    cuda.memcpy_dtod(d_dens_prev, d_dens, dens.nbytes)

    # Diffuse
    for _ in range(20):
        diffuse_kernel(d_dens, d_dens_prev, np.float32(diff), np.float32(dt), np.int32(0), np.int32(N),
                      block=block_2d, grid=grid_2d)
        set_bnd_kernel(d_dens, np.int32(0), np.int32(N), block=block_1d, grid=grid_1d)

    # Swap for advection
    cuda.memcpy_dtod(d_dens_prev, d_dens, dens.nbytes)

    # Advect
    advect_kernel(d_dens, d_dens_prev, d_u, d_v, np.float32(dt), np.int32(0), np.int32(N),
                 block=block_2d, grid=grid_2d)
    set_bnd_kernel(d_dens, np.int32(0), np.int32(N), block=block_1d, grid=grid_1d)

    # Reset sources for next step
    cuda.memset_d32(d_dens_prev, 0, N*N)

## 8. Utility Functions

In [293]:
def apply_force_at(x, y, dx, dy, density_amount, radius):
    # Apply force using the kernel
    apply_force_kernel(
        d_u, d_v, d_dens_prev,
        np.int32(x), np.int32(y),
        np.float32(dx * force), np.float32(dy * force),
        np.float32(density_amount),
        np.int32(radius), np.int32(N),
        block=block_2d, grid=grid_2d
    )

def clear_simulation():
    clear_fields_kernel(
        d_u, d_v, d_dens, np.int32(N),
        block=block_2d, grid=grid_2d
    )
    clear_fields_kernel(
        d_u_prev, d_v_prev, d_dens_prev, np.int32(N),
        block=block_2d, grid=grid_2d
    )

def get_density_for_display():
    # Get density data back from GPU
    cuda.memcpy_dtoh(dens, d_dens)
    # Return a copy to avoid modifying the original
    return dens.copy()

def get_velocity_for_display():
    # Get velocity data back from GPU
    cuda.memcpy_dtoh(u, d_u)
    cuda.memcpy_dtoh(v, d_v)
    # Compute magnitude of velocity
    vel_mag = np.sqrt(u*u + v*v)
    return vel_mag

def cleanup():
    """Free CUDA resources"""
    d_dens.free()
    d_dens_prev.free()
    d_u.free()
    d_v.free()
    d_u_prev.free()
    d_v_prev.free()
    d_p.free()
    d_div.free()

## 9. Color Functions

In [294]:
def get_density_color(value):
    """Get color for density value (0-1) with enhanced blue gradient"""
    if value < 0.05:
        return "#000033"      # Very dark blue
    elif value < 0.1:
        return "#000066"      # Dark blue
    elif value < 0.2:
        return "#0033cc"      # Medium dark blue
    elif value < 0.3:
        return "#0066ff"      # Medium blue
    elif value < 0.4:
        return "#3399ff"      # Bright blue
    elif value < 0.5:
        return "#66ccff"      # Light blue
    elif value < 0.6:
        return "#99ddff"      # Very light blue
    elif value < 0.7:
        return "#cceeaa"      # Light cyan-blue
    elif value < 0.8:
        return "#ffffcc"      # Light yellow
    elif value < 0.9:
        return "#ffcc99"      # Light orange
    else:
        return "#ff9999"      # Light red

def get_velocity_color(value):
    """Get color for velocity value (0-1) with enhanced rainbow gradient"""
    if value < 0.1:
        return "#004400"      # Dark green
    elif value < 0.2:
        return "#008800"      # Green
    elif value < 0.3:
        return "#44cc00"      # Yellow-green
    elif value < 0.4:
        return "#88ff00"      # Lime
    elif value < 0.5:
        return "#ccff00"      # Yellow-lime
    elif value < 0.6:
        return "#ffcc00"      # Yellow
    elif value < 0.7:
        return "#ff8800"      # Orange
    elif value < 0.8:
        return "#ff4400"      # Red-orange
    elif value < 0.9:
        return "#ff0000"      # Red
    else:
        return "#ff0088"      # Bright red-pink

## 10. Display and Visualization Functions

In [295]:
def fluid_to_rich_text(density, velocity, mode, cursor_pos):
    """Convert fluid data to rich text with enhanced colors and bold red cursor"""

    # Get maximum values for normalization
    max_dens = np.max(density)
    if max_dens == 0:
        max_dens = 1.0

    max_vel = np.max(velocity)
    if max_vel == 0:
        max_vel = 1.0

    # Get flow directions
    cuda.memcpy_dtoh(u, d_u)
    cuda.memcpy_dtoh(v, d_v)

    # Enhanced density characters for better gradient visualization
    density_chars = ' .·:;!|▪▫▪■'

    # Enhanced arrow characters with better visibility
    arrow_chars = ['→', '↘', '↓', '↙', '←', '↖', '↑', '↗']

    # Create the grid content as HTML with keyboard support and proper sizing
    html_content = f"""
    <div id="simulation-container" tabindex="0" style="
        outline: none;
        background: #111;
        border: 2px solid #444;
        border-radius: 8px;
        padding: 5px;
        margin: 5px 0;
        max-width: 100%;
        overflow: hidden;
    ">
        <pre id="simulation-grid" style='
            font-family: "Courier New", monospace;
            background: black;
            color: white;
            padding: 8px;
            font-size: 9px;
            line-height: 0.85;
            letter-spacing: 0.3px;
            white-space: pre;
            overflow: hidden;
            width: fit-content;
            max-width: 100%;
            box-sizing: border-box;
            margin: 0;
        '>"""

    for i in range(N):
        for j in range(N):
            # Enhanced cursor display with better visibility
            if cursor_pos and cursor_pos[0] == j and cursor_pos[1] == i:
                html_content += f"<span style='color: #ff0000; font-weight: bold; background: #444; border: 1px solid #ff0000;'>X</span>"
                continue

            # Determine what to display based on mode
            if mode == 'density':
                norm_dens = min(density[i, j] / max_dens, 1.0)
                if norm_dens < 0.01:
                    html_content += " "
                else:
                    idx = min(int(norm_dens * (len(density_chars) - 1)), len(density_chars)-1)
                    char = density_chars[idx]
                    color = get_density_color(norm_dens)
                    weight = "font-weight: bold;" if norm_dens > 0.7 else ""
                    html_content += f"<span style='color: {color}; {weight}'>{char}</span>"

            elif mode == 'velocity':
                norm_vel = min(velocity[i, j] / max_vel, 1.0)
                if norm_vel < 0.05:
                    html_content += " "
                else:
                    angle = np.arctan2(v[i, j], u[i, j])
                    dir_idx = int(((angle + np.pi) / (2 * np.pi) * 8 + 0.5) % 8)
                    char = arrow_chars[dir_idx]
                    color = get_velocity_color(norm_vel)
                    weight = "font-weight: bold;" if norm_vel > 0.6 else ""
                    html_content += f"<span style='color: {color}; {weight}'>{char}</span>"

            else:  # combined mode - show both with better logic
                norm_vel = min(velocity[i, j] / max_vel, 1.0)
                norm_dens = min(density[i, j] / max_dens, 1.0)

                # Prioritize showing velocity arrows when there's significant flow
                if norm_vel > 0.08:
                    angle = np.arctan2(v[i, j], u[i, j])
                    dir_idx = int(((angle + np.pi) / (2 * np.pi) * 8 + 0.5) % 8)
                    char = arrow_chars[dir_idx]
                    color = get_velocity_color(norm_vel)
                    weight = "font-weight: bold;" if norm_vel > 0.6 else ""
                    html_content += f"<span style='color: {color}; {weight}'>{char}</span>"
                elif norm_dens > 0.01:
                    # Show density when no significant velocity
                    idx = min(int(norm_dens * (len(density_chars) - 1)), len(density_chars)-1)
                    char = density_chars[idx]
                    color = get_density_color(norm_dens)
                    weight = "font-weight: bold;" if norm_dens > 0.7 else ""
                    html_content += f"<span style='color: {color}; {weight}'>{char}</span>"
                else:
                    html_content += " "

        html_content += "\n"

    html_content += """</pre>
        <div style="color: #888; font-size: 11px; text-align: center; padding: 5px; background: #222;">
            💡 Click here first, then use: ←→↑↓ move, Space=density, S=step, A=auto, C=clear, M=mode
        </div>
    </div>"""

    # Add status info with better formatting
    cursor_color = "#ff4444"
    mode_color = "#44ffff"
    auto_color = "#44ff44" if auto_update else "#ffff44"

    status_html = f"""
    <div style='
        font-family: monospace;
        background: linear-gradient(135deg, #1a1a1a, #2a2a2a);
        color: white;
        padding: 12px;
        margin-top: 8px;
        border-radius: 8px;
        border: 1px solid #444;
    '>
        <span style='color: {mode_color}; font-weight: bold; font-size: 14px;'>Mode:</span>
        <span style='color: white; font-size: 14px;'>{mode.capitalize()}</span> |
        <span style='color: {cursor_color}; font-weight: bold; font-size: 14px;'>Cursor:</span>
        <span style='color: white; font-size: 14px;'>({cursor_pos[0]}, {cursor_pos[1]})</span> |
        <span style='color: {auto_color}; font-weight: bold; font-size: 14px;'>Auto:</span>
        <span style='color: white; font-size: 14px;'>{'ON' if auto_update else 'OFF'}</span>
        <br>
        <span style='color: #aaa; font-size: 12px;'>Grid: {N}x{N} | Density Max: {max_dens:.3f} | Velocity Max: {max_vel:.3f}</span>
    </div>
    """

    # Title based on mode with better styling
    if mode == 'density':
        title = "💧 Fluid Density Visualization 💧"
        title_color = "#4169e1"
        subtitle = "Shows fluid concentration and distribution"
    elif mode == 'velocity':
        title = "🌊 Fluid Velocity Visualization 🌊"
        title_color = "#32cd32"
        subtitle = "Shows flow direction and speed"
    else:
        title = "🌀 Combined Visualization 🌀"
        title_color = "#da70d6"
        subtitle = "Shows both density and velocity flow"

    title_html = f"""
    <h3 style='
        color: {title_color};
        text-align: center;
        margin: 8px 0 4px 0;
        font-family: monospace;
        background: linear-gradient(135deg, #2a2a2a, #3a3a3a);
        padding: 12px;
        border-radius: 8px;
        border: 2px solid {title_color};
        box-shadow: 0 2px 8px rgba(0,0,0,0.3);
    '>{title}</h3>
    <p style='
        color: #ccc;
        text-align: center;
        margin: 0 0 8px 0;
        font-style: italic;
        font-size: 14px;
    '>{subtitle}</p>"""

    return title_html + html_content + status_html

def update_and_display():
    """Run simulation step and update display"""
    vel_step()
    dens_step()

    density = get_density_for_display()
    velocity = get_velocity_for_display()

    # Create HTML display instead of Rich text
    html_output = fluid_to_rich_text(density, velocity, display_mode, (cursor_x, cursor_y))

    # Display to output using HTML
    clear_output(wait=True)
    display(HTML(html_output))

## 11. Control Variables and Widget Setup

In [296]:
# Control variables
cursor_x, cursor_y = N // 2, N // 2
display_mode = 'combined'  # 'density', 'velocity', or 'combined'
running = True
auto_update = False

# Create widgets for controls
status_output = widgets.Output()
simulation_output = widgets.Output()
control_buttons = widgets.HBox([
    widgets.Button(description="Step", style=widgets.ButtonStyle(button_color='lightblue')),
    widgets.Button(description="Auto: OFF", style=widgets.ButtonStyle(button_color='lightgray')),
    widgets.Button(description="Clear", style=widgets.ButtonStyle(button_color='lightpink')),
    widgets.Button(description="Mode: Combined", style=widgets.ButtonStyle(button_color='lightgreen')),
    widgets.Button(description="Quit", style=widgets.ButtonStyle(button_color='salmon'))
])

force_controls = widgets.HBox([
    widgets.Button(description="←", layout=widgets.Layout(width='40px'),
                  style=widgets.ButtonStyle(button_color='lightblue')),
    widgets.Button(description="↑", layout=widgets.Layout(width='40px'),
                  style=widgets.ButtonStyle(button_color='lightblue')),
    widgets.Button(description="↓", layout=widgets.Layout(width='40px'),
                  style=widgets.ButtonStyle(button_color='lightblue')),
    widgets.Button(description="→", layout=widgets.Layout(width='40px'),
                  style=widgets.ButtonStyle(button_color='lightblue')),
    widgets.Button(description="•", layout=widgets.Layout(width='40px'),
                  style=widgets.ButtonStyle(button_color='gold'))
])

/usr/local/lib/python3.11/dist-packages/google/colab/_variable_inspector.py:27: UserWarning: module in out-of-thread context could not be cleaned up
  globals().clear()
/usr/local/lib/python3.11/dist-packages/google/colab/_variable_inspector.py:27: UserWarning: device_allocation in out-of-thread context could not be cleaned up
  globals().clear()


## 12. Keyboard Setup and Instructions

In [297]:
def setup_keyboard_buttons():
    """Simple button ID setup for keyboard events"""
    display(HTML("""
    <script>
    // Simple keyboard event handler
    document.addEventListener('keydown', function(event) {
        // Only trigger keyboard if no input elements are focused
        if (document.activeElement.tagName === 'INPUT' ||
            document.activeElement.tagName === 'TEXTAREA') {
            return;
        }

        console.log('Key pressed:', event.key);

        // Find and click the appropriate button
        var buttons = document.querySelectorAll('.widget-button');
        var buttonClicked = false;

        buttons.forEach(function(button) {
            var text = button.textContent.trim();

            switch(event.key) {
                case 'ArrowLeft':
                    if (text === '←') { button.click(); buttonClicked = true; }
                    break;
                case 'ArrowRight':
                    if (text === '→') { button.click(); buttonClicked = true; }
                    break;
                case 'ArrowUp':
                    if (text === '↑') { button.click(); buttonClicked = true; }
                    break;
                case 'ArrowDown':
                    if (text === '↓') { button.click(); buttonClicked = true; }
                    break;
                case ' ':
                    if (text === '•') {
                        event.preventDefault();
                        button.click();
                        buttonClicked = true;
                    }
                    break;
                case 's':
                case 'S':
                    if (text === 'Step') { button.click(); buttonClicked = true; }
                    break;
                case 'c':
                case 'C':
                    if (text === 'Clear') { button.click(); buttonClicked = true; }
                    break;
                case 'm':
                case 'M':
                    if (text.includes('Mode')) { button.click(); buttonClicked = true; }
                    break;
                case 'a':
                case 'A':
                    if (text.includes('Auto')) { button.click(); buttonClicked = true; }
                    break;
            }
        });

        // Prevent default for arrow keys and space
        if (['ArrowUp', 'ArrowDown', 'ArrowLeft', 'ArrowRight', ' '].includes(event.key) && buttonClicked) {
            event.preventDefault();
        }
    });

    // Focus handler for simulation container
    setTimeout(function() {
        var container = document.getElementById('simulation-container');
        if (container) {
            container.addEventListener('click', function() {
                container.focus();
                console.log('🎮 Keyboard controls activated!');
            });
        }
    }, 1000);
    </script>
    """))

# Instructions widget
instructions = widgets.HTML(
    value="""
    <h3 style="color:#6495ED;">Fluid Simulation Controls (80x80 Grid)</h3>
    <div style="background: #242424; padding: 15px; border-radius: 8px; margin: 10px 0;">
        <div style="display:flex; flex-direction:row; gap:20px;">
            <div style="flex:1;">
                <p><b style="color:#4CAF50;">Button Controls:</b></p>
                <ul style="margin: 5px 0;">
                    <li><b style="color:#2196F3;">Arrow buttons:</b> Move cursor & add force</li>
                    <li><b style="color:#2196F3;">Step:</b> Advance simulation one step</li>
                    <li><b style="color:#2196F3;">Auto:</b> Toggle automatic updates</li>
                    <li><b style="color:#2196F3;">Clear:</b> Reset simulation</li>
                    <li><b style="color:#2196F3;">Mode:</b> Change display mode</li>
                    <li><b style="color:#FFC107;">•:</b> Add density at cursor</li>
                </ul>
            </div>
            <div style="flex:1;">
                <p><b style="color:#4CAF50;">Keyboard Shortcuts:</b></p>
                <ul style="margin: 5px 0;">
                    <li><b style="color:#E91E63;">↑↓←→:</b> Move cursor & add force</li>
                    <li><b style="color:#E91E63;">Space:</b> Add density at cursor</li>
                    <li><b style="color:#E91E63;">S:</b> Step simulation</li>
                    <li><b style="color:#E91E63;">A:</b> Toggle auto mode</li>
                    <li><b style="color:#E91E63;">C:</b> Clear simulation</li>
                    <li><b style="color:#E91E63;">M:</b> Change mode</li>
                </ul>
                <p style="color: #FF5722; font-weight: bold; font-size: 14px;">
                    📝 Click on the simulation area to enable keyboard controls!
                </p>
            </div>
        </div>
    </div>
    """
)

## 13. Button Event Handlers

In [298]:
def on_step_click(b):
    with simulation_output:
        update_and_display()

def on_auto_click(b):
    global auto_update
    auto_update = not auto_update
    b.description = f"Auto: {'ON' if auto_update else 'OFF'}"
    b.style.button_color = 'lightgreen' if auto_update else 'lightgray'

    if auto_update:
        auto_update_simulation()

def on_clear_click(b):
    clear_simulation()
    with simulation_output:
        update_and_display()

def on_mode_click(b):
    global display_mode
    if display_mode == 'density':
        display_mode = 'velocity'
        b.style.button_color = 'lightgreen'
    elif display_mode == 'velocity':
        display_mode = 'combined'
        b.style.button_color = 'lavender'
    else:
        display_mode = 'density'
        b.style.button_color = 'lightcyan'

    b.description = f"Mode: {display_mode.capitalize()}"
    with simulation_output:
        update_and_display()

def on_quit_click(b):
    global running
    running = False
    cleanup()
    with status_output:
        console = Console(file=io.StringIO())
        console.print("[bold red]Simulation ended and resources cleaned up.[/]")
        print(console.file.getvalue())

def on_left_click(b):
    global cursor_x
    cursor_x = max(1, cursor_x - 1)
    apply_force_at(cursor_x, cursor_y, -2.0, 0.0, 20.0, 3)
    with simulation_output:
        update_and_display()

def on_right_click(b):
    global cursor_x
    cursor_x = min(N-2, cursor_x + 1)
    apply_force_at(cursor_x, cursor_y, 2.0, 0.0, 20.0, 3)
    with simulation_output:
        update_and_display()

def on_up_click(b):
    global cursor_y
    cursor_y = max(1, cursor_y - 1)
    apply_force_at(cursor_x, cursor_y, 0.0, -2.0, 20.0, 3)
    with simulation_output:
        update_and_display()

def on_down_click(b):
    global cursor_y
    cursor_y = min(N-2, cursor_y + 1)
    apply_force_at(cursor_x, cursor_y, 0.0, 2.0, 20.0, 3)
    with simulation_output:
        update_and_display()

def on_add_density(b):
    apply_force_at(cursor_x, cursor_y, 0.0, 0.0, 100.0, 5)
    with simulation_output:
        update_and_display()

# Connect button click handlers
control_buttons.children[0].on_click(on_step_click)
control_buttons.children[1].on_click(on_auto_click)
control_buttons.children[2].on_click(on_clear_click)
control_buttons.children[3].on_click(on_mode_click)
control_buttons.children[4].on_click(on_quit_click)

force_controls.children[0].on_click(on_left_click)
force_controls.children[1].on_click(on_up_click)
force_controls.children[2].on_click(on_down_click)
force_controls.children[3].on_click(on_right_click)
force_controls.children[4].on_click(on_add_density)

## 14. Auto-Update and Initial Content Functions

In [299]:
def auto_update_simulation():
    """Function to handle auto-updates"""
    if not running:
        return

    if auto_update:
        with simulation_output:
            update_and_display()

        # Schedule next update
        from IPython.display import display, Javascript
        display(Javascript("setTimeout(function() { IPython.notebook.kernel.execute('auto_update_simulation()'); }, 200);"))

def add_initial_vortex():
    """Add initial vortex with more particles in the center"""
    center_x, center_y = N // 2, N // 2
    radius = N // 3  # Larger radius for 80x80 grid

    # Add multiple vortices for more interesting flow
    vortices = [
        (center_x, center_y, radius, 1.0),           # Main central vortex
        (center_x - 15, center_y - 15, radius//2, -0.7),  # Counter-rotating vortex
        (center_x + 15, center_y + 15, radius//2, -0.7),  # Counter-rotating vortex
    ]

    for vortex_x, vortex_y, vortex_radius, strength in vortices:
        for i in range(N):
            for j in range(N):
                dx = i - vortex_x
                dy = j - vortex_y
                dist = np.sqrt(dx*dx + dy*dy)
                if dist < vortex_radius and dist > 1:
                    # Add circular flow
                    angle = np.arctan2(dy, dx)
                    force_x = -np.sin(angle) * strength * 3.0
                    force_y = np.cos(angle) * strength * 3.0
                    density_amount = 80.0 * (1 - dist/vortex_radius)
                    apply_force_at(i, j, force_x, force_y, density_amount, 2)

def add_random_particles():
    """Add random density particles across the grid"""
    import random

    for _ in range(20):  # Add 20 random particles
        x = random.randint(5, N-5)
        y = random.randint(5, N-5)
        density_amount = random.uniform(30, 80)
        apply_force_at(x, y, 0, 0, density_amount, 3)

## 15. Automated Random Movement Generation

In [300]:
import random
import time
from IPython.display import Javascript

class RandomMovementGenerator:
    def __init__(self, total_frames=200, delay_ms=100):
        self.total_frames = total_frames
        self.delay_ms = delay_ms
        self.current_frame = 0
        self.is_running = False
        self.current_pattern = None
        self.movement_patterns = [
            'random_walk',
            'spiral',
            'figure_eight',
            'bouncing_ball',
            'vortex_dance'
        ]

    def random_walk(self):
        """Random walk pattern"""
        global cursor_x, cursor_y
        directions = [(-1, 0), (1, 0), (0, -1), (0, 1), (-1, -1), (1, 1), (-1, 1), (1, -1)]
        dx, dy = random.choice(directions)

        new_x = max(1, min(N-2, cursor_x + dx))
        new_y = max(1, min(N-2, cursor_y + dy))
        cursor_x, cursor_y = new_x, new_y

        # Add random force with density
        force_magnitude = random.uniform(1.0, 3.0)
        density_amount = random.uniform(20.0, 60.0)
        apply_force_at(cursor_x, cursor_y, dx * force_magnitude, dy * force_magnitude, density_amount, random.randint(2, 5))

    def spiral(self):
        """Spiral pattern"""
        global cursor_x, cursor_y
        center_x, center_y = N // 2, N // 2
        angle = (self.current_frame * 0.3) % (2 * np.pi)
        radius = min(20, self.current_frame * 0.1)

        new_x = int(center_x + radius * np.cos(angle))
        new_y = int(center_y + radius * np.sin(angle))

        cursor_x = max(1, min(N-2, new_x))
        cursor_y = max(1, min(N-2, new_y))

        # Add tangential force
        force_x = -np.sin(angle) * 2.0
        force_y = np.cos(angle) * 2.0
        apply_force_at(cursor_x, cursor_y, force_x, force_y, 40.0, 3)

    def figure_eight(self):
        """Figure-eight pattern"""
        global cursor_x, cursor_y
        center_x, center_y = N // 2, N // 2
        t = self.current_frame * 0.2

        # Parametric equations for figure-eight
        scale = 15
        new_x = int(center_x + scale * np.sin(t))
        new_y = int(center_y + scale * np.sin(t) * np.cos(t))

        cursor_x = max(1, min(N-2, new_x))
        cursor_y = max(1, min(N-2, new_y))

        # Add force in direction of movement
        force_x = np.cos(t) * 2.0
        force_y = (np.cos(2*t)) * 2.0
        apply_force_at(cursor_x, cursor_y, force_x, force_y, 50.0, 4)

    def bouncing_ball(self):
        """Bouncing ball pattern"""
        global cursor_x, cursor_y
        if not hasattr(self, 'ball_vx'):
            self.ball_vx = random.uniform(-2, 2)
            self.ball_vy = random.uniform(-2, 2)
            self.ball_x = float(cursor_x)
            self.ball_y = float(cursor_y)

        # Update position
        self.ball_x += self.ball_vx
        self.ball_y += self.ball_vy

        # Bounce off walls
        if self.ball_x <= 1 or self.ball_x >= N-2:
            self.ball_vx *= -0.9  # Add some damping
        if self.ball_y <= 1 or self.ball_y >= N-2:
            self.ball_vy *= -0.9

        # Clamp to bounds
        self.ball_x = max(1, min(N-2, self.ball_x))
        self.ball_y = max(1, min(N-2, self.ball_y))

        cursor_x = int(self.ball_x)
        cursor_y = int(self.ball_y)

        apply_force_at(cursor_x, cursor_y, self.ball_vx, self.ball_vy, 60.0, 5)

    def vortex_dance(self):
        """Multiple vortex dance pattern"""
        global cursor_x, cursor_y

        # Create multiple moving vortices
        num_vortices = 3
        for i in range(num_vortices):
            angle = (self.current_frame * 0.1 + i * 2 * np.pi / num_vortices) % (2 * np.pi)
            radius = 15 + 10 * np.sin(self.current_frame * 0.05 + i)

            center_x = N // 2 + int(radius * np.cos(angle))
            center_y = N // 2 + int(radius * np.sin(angle))

            # Clamp to bounds
            vortex_x = max(5, min(N-5, center_x))
            vortex_y = max(5, min(N-5, center_y))

            # Add vortex forces
            vortex_angle = angle + np.pi/2  # Perpendicular for rotation
            force_x = np.cos(vortex_angle) * 3.0
            force_y = np.sin(vortex_angle) * 3.0

            apply_force_at(vortex_x, vortex_y, force_x, force_y, 30.0, 3)

        # Move cursor to follow one of the vortices
        cursor_x = max(1, min(N-2, N // 2 + int(15 * np.cos(self.current_frame * 0.1))))
        cursor_y = max(1, min(N-2, N // 2 + int(15 * np.sin(self.current_frame * 0.1))))

    def execute_pattern(self, pattern_name):
        """Execute a specific movement pattern"""
        if pattern_name == 'random_walk':
            self.random_walk()
        elif pattern_name == 'spiral':
            self.spiral()
        elif pattern_name == 'figure_eight':
            self.figure_eight()
        elif pattern_name == 'bouncing_ball':
            self.bouncing_ball()
        elif pattern_name == 'vortex_dance':
            self.vortex_dance()

    def run_random_sequence(self, pattern=None):
        """Run a sequence of random movements"""
        if self.is_running:
            print("⚠️ Animation already running! Use Stop button first.")
            return

        self.is_running = True
        self.current_frame = 0

        # Reset ball physics for bouncing pattern
        if hasattr(self, 'ball_vx'):
            delattr(self, 'ball_vx')
            delattr(self, 'ball_vy')
            delattr(self, 'ball_x')
            delattr(self, 'ball_y')

        # Choose pattern
        if pattern is None:
            pattern = random.choice(self.movement_patterns)

        self.current_pattern = pattern
        print(f"🎬 Starting {self.total_frames} frames of '{pattern}' pattern...")

        # Start the animation loop
        self.continue_animation()

    def continue_animation(self):
        """Continue the animation (called recursively)"""
        if not self.is_running or self.current_frame >= self.total_frames:
            self.is_running = False
            print(f"✅ Animation complete! ({self.current_frame} frames)")
            return

        # Execute movement pattern
        if self.current_pattern:
            self.execute_pattern(self.current_pattern)

        # Update simulation and display
        with simulation_output:
            update_and_display()

        self.current_frame += 1
        print(f"Frame {self.current_frame}/{self.total_frames}")  # Debug output

        # Schedule next frame if not finished
        if self.current_frame < self.total_frames and self.is_running:
            # Use a more reliable approach with direct kernel execution
            import threading
            import time

            def delayed_call():
                time.sleep(self.delay_ms / 1000.0)  # Convert ms to seconds
                if self.is_running:  # Check if still running
                    self.continue_animation()

            thread = threading.Thread(target=delayed_call)
            thread.daemon = True
            thread.start()
        else:
            self.is_running = False
            print(f"✅ Animation complete! ({self.current_frame} frames)")

    def stop(self):
        """Stop the current animation"""
        if self.is_running:
            self.is_running = False
            print(f"🛑 Animation stopped at frame {self.current_frame}.")
        else:
            print("ℹ️ No animation running.")

# Alternative simpler animation function that works better in Colab
def run_simple_animation(pattern_name, frames=50):
    """Run a simple animation without threading (more reliable in Colab)"""
    print(f"🎬 Running {frames} frames of {pattern_name}...")

    # Create a temporary generator
    temp_generator = RandomMovementGenerator()
    temp_generator.current_pattern = pattern_name

    for frame in range(frames):
        temp_generator.current_frame = frame
        temp_generator.execute_pattern(pattern_name)

        with simulation_output:
            update_and_display()

        print(f"Frame {frame + 1}/{frames}")
        time.sleep(0.1)  # Simple delay

    print(f"✅ Simple animation complete!")

# Create global random movement generator
random_generator = RandomMovementGenerator(total_frames=200, delay_ms=150)

# Create control buttons for random movement
random_control_buttons = widgets.HBox([
    widgets.Button(description="🎲 Random Walk", style=widgets.ButtonStyle(button_color='lightcyan')),
    widgets.Button(description="🌀 Spiral", style=widgets.ButtonStyle(button_color='lightgreen')),
    widgets.Button(description="∞ Figure-8", style=widgets.ButtonStyle(button_color='lightyellow')),
    widgets.Button(description="⚽ Bouncing", style=widgets.ButtonStyle(button_color='lightpink')),
    widgets.Button(description="🌪️ Vortex Dance", style=widgets.ButtonStyle(button_color='lavender')),
    widgets.Button(description="🎯 Random Pattern", style=widgets.ButtonStyle(button_color='lightgray')),
    widgets.Button(description="⏹️ Stop", style=widgets.ButtonStyle(button_color='salmon'))
])

# Button handlers for random movement - using simple animation
def on_random_walk_click(b):
    run_simple_animation('random_walk', 100)

def on_spiral_click(b):
    run_simple_animation('spiral', 100)

def on_figure_eight_click(b):
    run_simple_animation('figure_eight', 100)

def on_bouncing_click(b):
    run_simple_animation('bouncing_ball', 100)

def on_vortex_dance_click(b):
    run_simple_animation('vortex_dance', 100)

def on_random_pattern_click(b):
    pattern = random.choice(random_generator.movement_patterns)
    run_simple_animation(pattern, 100)

def on_stop_animation_click(b):
    random_generator.stop()
    print("🛑 Animation controls stopped.")

# Connect random movement button handlers
random_control_buttons.children[0].on_click(on_random_walk_click)
random_control_buttons.children[1].on_click(on_spiral_click)
random_control_buttons.children[2].on_click(on_figure_eight_click)
random_control_buttons.children[3].on_click(on_bouncing_click)
random_control_buttons.children[4].on_click(on_vortex_dance_click)
random_control_buttons.children[5].on_click(on_random_pattern_click)
random_control_buttons.children[6].on_click(on_stop_animation_click)

# Instructions for random movement
random_instructions = widgets.HTML(
    value="""
    <h4 style="color:#9C27B0;">🎬 Automated Movement Patterns (100 frames each)</h4>
    <div style="background: #2a2a2a; padding: 10px; border-radius: 6px; margin: 5px 0; color: white;">
        <div style="display:flex; flex-direction:row; gap:15px; font-size: 12px;">
            <div style="flex:1;">
                <b style="color:#FF9800;">Movement Patterns:</b>
                <ul style="margin: 3px 0; padding-left: 15px;">
                    <li><b>🎲 Random Walk:</b> Chaotic movement with random forces</li>
                    <li><b>🌀 Spiral:</b> Expanding spiral motion</li>
                    <li><b>∞ Figure-8:</b> Smooth figure-eight pattern</li>
                </ul>
            </div>
            <div style="flex:1;">
                <b style="color:#FF9800;">Advanced Patterns:</b>
                <ul style="margin: 3px 0; padding-left: 15px;">
                    <li><b>⚽ Bouncing:</b> Physics-based bouncing ball</li>
                    <li><b>🌪️ Vortex Dance:</b> Multiple moving vortices</li>
                    <li><b>🎯 Random Pattern:</b> Picks a random pattern</li>
                </ul>
            </div>
        </div>
        <p style="margin: 5px 0; font-size: 11px; color: #aaa;">Each pattern runs for 100 frames automatically. Simple but reliable implementation.</p>
    </div>
    """
)

print("🎬 Random Movement Generator loaded! Ready for automated animations.")

🎬 Random Movement Generator loaded! Ready for automated animations.


## 16. Display Widgets and Initialize Simulation

In [301]:
# Display widgets
display(instructions)
display(simulation_output)
display(control_buttons)
display(force_controls)
display(random_instructions)  # Add this line
display(random_control_buttons)  # Add this line
display(status_output)

# Setup and initialize
add_initial_vortex()
add_random_particles()
setup_keyboard_buttons()

with simulation_output:
    update_and_display()

with status_output:
    console = Console(file=io.StringIO(), color_system="truecolor")
    console.print("[bold green]Rich Fluid Simulation Started[/]")
    console.print("[yellow]Use the buttons to interact with the simulation[/]")
    print(console.file.getvalue())

# Register cleanup function
import atexit
atexit.register(cleanup)

HTML(value='\n    <h3 style="color:#6495ED;">Fluid Simulation Controls (80x80 Grid)</h3>\n    <div style="back…

Output()

HTML(value='\n    <h4 style="color:#9C27B0;">🎬 Automated Movement Patterns (100 frames each)</h4>\n    <div st…

Output()

<function __main__.cleanup()>

<IPython.core.display.Javascript object>

🎬 Running 100 frames of spiral...
Frame 1/100
Frame 2/100
Frame 3/100
Frame 4/100
Frame 5/100
Frame 6/100
Frame 7/100
Frame 8/100
Frame 9/100
Frame 10/100
Frame 11/100
Frame 12/100
Frame 13/100
Frame 14/100
Frame 15/100
Frame 16/100
Frame 17/100
Frame 18/100
Frame 19/100
Frame 20/100
Frame 21/100
Frame 22/100
Frame 23/100
Frame 24/100
Frame 25/100
Frame 26/100
Frame 27/100
Frame 28/100
Frame 29/100
Frame 30/100
Frame 31/100
Frame 32/100
Frame 33/100
Frame 34/100
Frame 35/100
Frame 36/100
Frame 37/100
Frame 38/100
Frame 39/100
Frame 40/100
Frame 41/100
Frame 42/100
Frame 43/100
Frame 44/100
Frame 45/100
Frame 46/100
Frame 47/100
Frame 48/100
Frame 49/100
Frame 50/100
Frame 51/100
Frame 52/100
Frame 53/100
Frame 54/100
Frame 55/100
Frame 56/100
Frame 57/100
Frame 58/100
Frame 59/100
Frame 60/100
Frame 61/100
Frame 62/100
Frame 63/100
Frame 64/100
Frame 65/100
Frame 66/100
Frame 67/100
Frame 68/100
Frame 69/100
Frame 70/100
Frame 71/100
Frame 72/100
Frame 73/100
Frame 74/100
Frame 75/100
